<a href="https://colab.research.google.com/github/TatanPerez/Senales_Sistemas/blob/main/Dashboard_punto_4_parcial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit -q #instalación de librerías
!pip install pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.2 MB/s eta 0:00:00


In [ ]:
%%writefile THD.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
st.sidebar.title("Involucrados en el Proyecto")
st.sidebar.write("Juan Jeronimo Castaño Rivera")
st.sidebar.write("Daniel Mauricio Mejia Hoyos")
st.sidebar.write("Wilmer Sebastian Perez Cuastumal")
# Título y descripción general
st.title('Cálculo del THD en Circuitos RC y Resistivos Puros')
st.write("""
En este dashboard se muestra el cálculo de la **Distorsión Armónica Total (THD)** en circuitos RC y resistivos puros.
""")

st.write("""
### ¿Qué es el THD?
La **Distorsión Armónica Total (THD)** es una medida de la cantidad de distorsión armónica presente en una señal.
Se calcula comparando la magnitud de los armónicos superiores con la magnitud de la frecuencia fundamental.

El **THD** se define como:
""")

st.latex("THD = \\frac{\\sqrt{V_2^2 + V_3^2 + V_4^2 + ...}}{V_1}")

st.write("""
Donde: """)
st.latex("V_1")
st.write("""es la magnitud de la frecuencia fundamental y: """)
st.latex("V_2, V_3, V_4,...")
st.write("""son las magnitudes de los armónicos superiores.
""")

# Explicación del proceso de cálculo del THD usando FFT
st.write("""
### Cálculo del THD a partir de la FFT
Para calcular el THD, se uasa la **Transformada Rápida de Fourier (FFT)**, que permite transformar una señal del dominio del tiempo al dominio de la frecuencia.

Primero, se obtiene la **frecuencia fundamental** de la señal, que es el componente de mayor magnitud en el espectro de frecuencias. Luego, se calcula la contribución de los armónicos (frecuencias superiores) y con la fórmula anterior se obtiene el THD.
""")

# Parámetros de la señal de entrada
Fo = 60  # Frecuencia de la señal (Hz)
Fs = 30 * Fo  # Frecuencia de muestreo (Hz)
To = 1 / Fo  # Periodo fundamental
Ts = 1 / Fs  # Periodo de muestreo
t = np.arange(0, 5 * To, Ts)  # Simulamos 5 periodos de la señal
A = 120  # Amplitud de la señal de entrada

# Generación de la señal de entrada y su rectificación
st.subheader("Señal de Entrada y Rectificación")
st.write("""
Se genera una señal senoidal de 60 Hz, que es típica en sistemas de alimentación eléctrica.
Luego, se rectifica para simular la señal que pasa por un circuito de rectificación.
""")
in_o = A * np.sin(2 * np.pi * Fo * t)
rec_c = sig.square(2 * np.pi * Fo * t)  # Tren de pulsos para la señal rectificada completa
in_ = in_o * rec_c  # Entrada rectificada completa

# Mostrar las gráficas de la señal de entrada y rectificada
st.write("### Señal de Alimentación Original (Senoidal)")
plt.figure()
plt.plot(t, in_o, label='Alimentación')
plt.xlabel('Tiempo [s]')
plt.ylabel('Amplitud')
plt.legend()
st.pyplot(plt)

st.write("### Señal Rectificada Completa")
plt.figure()
plt.plot(t, in_, label='Señal Rectificada')
plt.xlabel('Tiempo [s]')
plt.ylabel('Amplitud')
plt.legend()
st.pyplot(plt)

# Slider para ajustar R y C
st.subheader("Parámetros del Circuito RC")
st.write("""
Ahora se pueden modificar los valores de la resistencia (R) y la capacitancia (C) para observar cómo afecta el circuito RC a la señal rectificada.
""")
R_v = st.slider('Resistencia R (Ohmios)', min_value=100, max_value=2000, value=1000, step=100)
C_v = st.slider('Capacitancia C (μF)', min_value=1, max_value=50, value=10, step=1) * 1e-6

# Función de transferencia del circuito RC en serie
st.write("""
### Función de Transferencia del Circuito RC en Serie
Para el circuito RC en serie, la función de transferencia se define como:
""")
st.latex("H(s) = \\frac{1}{RCs + 1}")


st.write("""
Esta función de transferencia permite modelar el comportamiento del circuito en el dominio de la frecuencia.
""")
# Definición de la función de transferencia del circuito RC
num = np.array([1])  # Coeficiente del numerador
den = np.array([R_v * C_v, 1])  # Coeficientes del denominador
G_n = sig.TransferFunction(num, den)  # Definición de la función de transferencia

# Evaluar la salida del circuito RC
_, out_rc, _ = sig.lsim(G_n, U=in_, T=t)

# Gráfica de la salida del circuito RC
st.subheader("Salida del Circuito RC")
st.write("""
La siguiente gráfica muestra cómo la señal rectificada cambia al pasar por el circuito RC en serie.
""")
plt.figure()
plt.plot(t, out_rc, label='Salida del Circuito RC')
plt.xlabel('Tiempo [s]')
plt.ylabel('Amplitud')
plt.legend()
st.pyplot(plt)
#plt.plot(t, in_,label='In(t)')#entrada rectificada
st.subheader("Salida del Circuito resistivo")
st.write("""
La siguiente gráfica muestra cómo la señal rectificada de salida del circuito resistivo.
""")
plt.figure()
plt.plot(t, in_,c="r",label='Out(t)')#salida del circuito
plt.xlabel('t [s]')
plt.ylabel('out(t)')
st.pyplot(plt)
# Cálculo del THD simplificado basado en la FFT
def calcular_thd(signal, Fs):
    Xf = np.fft.rfft(signal)  # FFT de la señal
    magnitudes = np.abs(Xf)
    freqs = np.fft.rfftfreq(len(signal), 1 / Fs)

    # Identificar la frecuencia fundamental
    fundamental_idx = np.argmax(magnitudes)
    fundamental = magnitudes[fundamental_idx]

    # Cálculo del THD como la suma de los armónicos sobre la fundamental
    thd = np.sqrt(np.sum(magnitudes[1:]**2)) / fundamental
    return thd

# Calcular el THD
thd_rc = calcular_thd(out_rc, Fs)
thd_resistive = calcular_thd(in_, Fs)

# Mostrar el valor del THD
st.subheader("Resultados del THD")
st.markdown("""
El valor del THD es una medida de la distorsión de la señal debida a los armónicos.
A continuación, se muestran los valores del THD tanto para el circuito resistivo puro como para el circuito RC en serie:
""")
st.markdown(f"**THD del circuito RC**: {thd_rc * 100:.2f}%")
st.markdown(f"**THD del circuito resistivo puro**: {thd_resistive * 100:.2f}%")
st.subheader('Cálculo del Factor de Potencia basado en THD')

st.subheader('¿Qué es el Factor de Potencia?')
st.write('''
El **Factor de Potencia (FP)** mide la eficiencia con la que un sistema eléctrico convierte la energía eléctrica en trabajo útil.
Cuando la señal tiene **Distorsión Armónica Total (THD)**, parte de la potencia se pierde en los armónicos, reduciendo la eficiencia.

La fórmula general del Factor de Potencia con THD es:
''')

st.latex(r'''
FP = \sqrt{\frac{1}{1 + \left(THD\right)^2}}
''')

st.write('''
Donde el **THD** representa el porcentaje de distorsión de la señal. A medida que aumenta el **THD**, más potencia se pierde en los armónicos, reduciendo el **Factor de Potencia**.
''')
st.markdown("""
A continuación, se muestran los valores del factor de potencia tanto para el circuito resistivo puro como para el circuito RC en serie:
""")
pf1 = np.sqrt(1 / (1 + (thd_rc )**2))
pf2 = np.sqrt(1 / (1 + (thd_resistive)**2))
st.markdown(f"**FP del circuito RC**: {pf1:.4f}")
st.markdown(f"**FP del circuito resistivo puro**: {pf2:.4f}")

# FFT de la señal de salida para el circuito RC
vfre_rc = np.fft.rfftfreq(len(out_rc), Ts)  # Vector de frecuencias en Hz
Xf_rc = np.fft.rfft(out_rc)  # FFT de la salida

st.markdown("### FFT de la Salida del Circuito RC")
st.markdown("""
La **Transformada de Fourier** nos muestra el contenido de frecuencias de la señal.
La siguiente gráfica muestra cómo se distribuyen las frecuencias en la señal después de pasar por el circuito RC.
""")
plt.figure()
plt.stem(vfre_rc, np.abs(Xf_rc) / len(out_rc), use_line_collection=True)
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Magnitud FFT')
st.pyplot(plt)

# FFT de la señal para el circuito resistivo puro
vfre_resistive = np.fft.rfftfreq(len(in_), Ts)
Xf_resistive = np.fft.rfft(in_)

st.write("### FFT del Circuito Resistivo Puro")
st.write("""
Este gráfico muestra la FFT de la señal en un circuito resistivo puro, sin ningún circuito adicional que altere la señal.
""")
plt.figure()
plt.stem(vfre_resistive, np.abs(Xf_resistive) / len(in_), use_line_collection=True)
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Magnitud FFT')
st.pyplot(plt)



Overwriting THD.py


In [ ]:
token = '2mQxb0qST26pDiDOlaLUjZoa6Qz_SHb63A9scmGKAGMN2E65' #colocar aquí su token personal después de crear su cuenta con correo UNAL en Ngrok

In [ ]:
from pyngrok import ngrok

# Set authentication token (unique per user)
ngrok.set_auth_token(token)

# Start Streamlit server on a specific port
!nohup streamlit run THD.py --server.port 5011 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://4af4-34-90-148-130.ngrok-free.app
